### 행전안전부 보도자료
- 뉴스,소식 > 보도자료 데이터 수집
- `https://mois.go.kr/`
- 정적(static) 웹페이지 데이수집 
- post 방식의 데이터 수집
- 여러페이지 데이터 수집

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [2]:
# 1. 웹서비스 분석 : URL

In [3]:
page = 5
url = "https://mois.go.kr/frt/bbs/type010/commonSelectBoardList.do?bbsId=BBSMSTR_000000000008"
params = {
    "nttId": "0",
    "bbsTyCode": "BBST03",
    "bbsAttrbCode": "BBSA03",
    "authFlag": "Y",
    "pageIndex": page,
    "cal_url": "/sym/cal/EgovNormalCalPopup.do",
    "searchCnd": "0",
}

In [4]:
# 2. request > response : str(html)

In [5]:
response = requests.post(url, params)
response

<Response [200]>

In [6]:
response.text[:250]

'<!DOCTYPE html>\r\n<html lang="ko">\r\n<head>\r\n<title>\r\n\r\n행정안전부&gt;  뉴스·소식&gt; 보도자료&gt; 보도자료</title>\r\n<meta http-equiv="X-UA-Compatible" content="IE=edge" />\r\n<meta charset="utf-8" />\r\n<meta name="viewport" content="width=device-width,maximum-scale=1,min'

In [7]:
# 3. bs object > select() : text(str)

In [8]:
dom = BeautifulSoup(response.text, "html.parser")

In [9]:
selector = '#print_area > div.table_wrap.type_01 > form > table > tbody > tr'
elements = dom.select(selector)
len(elements)

10

In [10]:
element = elements[0]

In [11]:
data = {
    "no": element.select("td")[0].text.strip(),
    "title": element.select("td")[1].text.strip(),
    "link": "https://mois.go.kr" + element.select("td")[1].select_one("a").get("href"),
    "writer": element.select("td")[3].text.strip(),
    "date": element.select("td")[4].text.strip(),
    "pv": element.select("td")[5].text.strip(),
}
data

{'no': '12883',
 'title': '정부 과천청사관리소, 「상호 존중의 날」 행사 개최',
 'link': 'https://mois.go.kr/frt/bbs/type010/commonSelectBoardArticle.do;jsessionid=pixSbNIHZeaj9WBJySm4wuZb.node50?bbsId=BBSMSTR_000000000008&nttId=90097',
 'writer': '관리과',
 'date': '2022.01.24.',
 'pv': '572'}

In [12]:
datas = []
for element in elements:
    datas.append({
        "no": element.select("td")[0].text.strip(),
        "title": element.select("td")[1].text.strip(),
        "link": "https://mois.go.kr" + element.select("td")[1].select_one("a").get("href"),
        "writer": element.select("td")[3].text.strip(),
        "date": element.select("td")[4].text.strip(),
        "pv": element.select("td")[5].text.strip(),
    })

In [13]:
# 4. DataFrame

In [14]:
df = pd.DataFrame(datas)
df.tail(2)

,no,title,link,writer,date,pv
8,12875,"서해 5도 주민, 정주생활지원금 월 12만 원으로 20% 인상",https://mois.go.kr/frt/bbs/type010/commonSelec...,지역균형발전과,2022.01.19.,779
9,12874,중앙-지자체 주소기반 신사업 개발 위해 머리 맞댄다,https://mois.go.kr/frt/bbs/type010/commonSelec...,주소정책과,2022.01.19.,813


In [15]:
# 5. 함수로 만들어서 1페이지 ~ 3페이지까지 데이터 수집

In [16]:
def mois(page):
    
    url = "https://mois.go.kr/frt/bbs/type010/commonSelectBoardList.do?bbsId=BBSMSTR_000000000008"
    params = {
        "nttId": "0",
        "bbsTyCode": "BBST03",
        "bbsAttrbCode": "BBSA03",
        "authFlag": "Y",
        "pageIndex": page,
        "cal_url": "/sym/cal/EgovNormalCalPopup.do",
        "searchCnd": "0",
    }
    response = requests.post(url, params)
    dom = BeautifulSoup(response.text, "html.parser")
    selector = '#print_area > div.table_wrap.type_01 > form > table > tbody > tr'
    elements = dom.select(selector)
    
    datas = []
    for element in elements:
        datas.append({
            "no": element.select("td")[0].text.strip(),
            "title": element.select("td")[1].text.strip(),
            "link": "https://mois.go.kr" + element.select("td")[1].select_one("a").get("href"),
            "writer": element.select("td")[3].text.strip(),
            "date": element.select("td")[4].text.strip(),
            "pv": element.select("td")[5].text.strip(),
        })
    return pd.DataFrame(datas)

In [17]:
dfs = []
start, end = 1, 3
for page in range(start, end + 1):
    print(page, end=" ")
    dfs.append(mois(page))

1 2 3 

In [18]:
result = pd.concat(dfs, ignore_index=True)
result.tail(2)

,no,title,link,writer,date,pv
28,12895,정부세종청사 중앙동 미술작품 공모 당선작 선정,https://mois.go.kr/frt/bbs/type010/commonSelec...,청사건축과,2022.01.26.,581
29,12894,"설 명절, 교통사고와 주택화재에 주의하세요!",https://mois.go.kr/frt/bbs/type010/commonSelec...,홍보담당관,2022.01.26.,645
